In [1]:
from keybert import KeyBERT
from kiwipiepy import Kiwi
from transformers import BertModel
from tqdm import tqdm
import pandas as pd
model = BertModel.from_pretrained('skt/kobert-base-v1')
kw_model = KeyBERT(model)

# Load the CSV file again
data = pd.read_csv('preprocessed_content_market.csv')

# Extracting each row from the 'preprocessed_content' column and storing it in a list
content_list = data['preprocessed_content'].tolist()

# Displaying the first few items in the list to verify
content_list[:2]

/home/winten/.pyenv/versions/3.10.11/envs/langchain_kurly/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['리뷰 마켓 컬리 8 주년 위 크 역대 급 특 가 추천 상품 포도 계란 소고기 마트 보다 저렴하게 고 미리 레 2023 5 22 12 24 url 복사 이웃 추가 본문 기타 기능 마켓 컬리 8 주년 위 크 역대 급 특 가 추천 상품 포도 계란 소고기 마트 보다 저렴하게 안녕하세요 네이버 인 플루 언 서 고 미리 레 입니다 제 가 너무나 애정 하고 있는 마켓 컬리 가 벌써 8 주년 되었더라고요 일주일 에 2 3일 은 이용 하고 있어서 이제 는 없어서는 안되는 필수 앱 되었답니다 이번 8 주년 맞아 마켓 컬리 에서 준비 한 다양한 역대 급 특 가 할인 상품 들 나왔는데요 거기 에 카드 할인 까지 더해 저렴하게 구입 할 수 있는 정보 알려 드릴게요 컬리 8 th birth week 이벤트 소식 및 기간 이번 마켓 컬리 8 주년 위 크는 단 일주일 간 진행 되는 이벤트 인데 요 5월 22일 월요일 오전 11시 부터 5월 28일 일요일 오전 11시 까지만 이용 하실 수 있어요 반값특 가 상품 은 물론 매일 다른 특 가 상품 만 나볼 수 있는 일일 특 가도 있으니 이번 기회 놓치지 말아야겠죠 밖 매일 식탁 위 에 올라가는 상품 쟁여둘 수록 이득 인 쟁여템 만날 수 있는 장바구니 특가 컬리 가 만든 고 퀄리티 상 컬리 온리 특가 도 있으니 하나 씩 소개 해 드릴게요 마켓 컬리 첫 구매 친구 추천 쿠폰 5 000원 가입 시 입력 하시고 추가 할인 받으세요 쿠폰 코드 market 5000 컬리 첫 구 매 시 10 000원 쿠폰 받으러 가기 아무래도 마켓 컬리 장점 은 다양한 식 자재 상품 들 빠르게 새벽 에 배송 받을 수 있다는 점 죠 그래서 바로바로 필요한 채소 나 먹거리 들 아침 부터 잘 챙겨 먹을 수 있어 특히 바쁜 주부 분들 이나 직장인 분들 필수 앱 되어 버린 같아요 할인 하고 있는 상품 들 도 빠르게 접 할 수 있어 매일 마켓 컬리 앱 살펴보게 되더라고요 마켓 컬리 자체 브랜드 도 저렴하고 퀄리티 도 높아서 자주 사용 하고 있답니다 마켓 컬리 8 주년 세 일 추천

In [3]:
# Kiwi와 KeyBERT 모델 초기화
kiwi = Kiwi()
kw_model = KeyBERT()

# 명사 추출 함수
def noun_extractor(text):
    results = []
    result = kiwi.analyze(text)
    for token, pos, _, _ in result[0][0]:
        if len(token) != 1 and (pos.startswith('N') or pos.startswith('SL')):
            results.append(token)
    return results

# 각 행의 텍스트에 대해 명사 추출 및 키워드 추출
extracted_keywords = []
for text in tqdm(content_list):
    # 명사 추출
    nouns = noun_extractor(text)
    after_kiwi_text = ' '.join(nouns)
    
    # KeyBERT를 사용한 키워드 추출
    preprocessed_content_keywords = kw_model.extract_keywords(after_kiwi_text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=3)
    
    # 추출된 키워드 저장
    extracted_keywords.append(preprocessed_content_keywords)

# 결과 출력
print(extracted_keywords[:5])


 12%|█▏        | 476/3993 [00:34<04:05, 14.33it/s]

In [ ]:
# Extracting only the words from the extracted keywords
words_only = [[keyword[0] for keyword in keywords_list] for keywords_list in extracted_keywords]

# Creating a new DataFrame with the words only
keywords_df = pd.DataFrame({
    'Extracted_Keywords': words_only
})

keywords_df.head()

,Extracted_Keywords
0,"[컬리, 소고기, 애정]"
1,"[냉동식품, 분식집, 국민학교]"
2,"[컬리, 키트, 유저]"
3,"[컬리, 어시장, 모듬]"
4,"[충성, 쿠폰, 컬리]"


In [ ]:
importance_df = pd.DataFrame()
importance_df['importance'] = data['likes'].values + data['comments'].values
importance_keyword_df = pd.concat([importance_df,keywords_df],axis=1)
datetime_keyword_hashtags_importance_df = pd.concat([data[['datetime','hashtags']],importance_keyword_df], axis=1)
datetime_keyword_hashtags_importance_df.head()

,importance
0,56.0
1,6.0
2,83.0
3,6.0
4,10.0


In [22]:
datetime_keyword_hashtags_importance_df.to_csv('datetime_keyword_hashtags_importance_market.csv', encoding='utf-8', index=False)